In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
# 경로 설정
train_dir = 'C:/Users/intel/Downloads/trainset(300x300)'
test_dir = 'C:/Users/intel/Downloads/testset(300x300)'

# ImageDataGenerator 설정 (데이터 증강 없이 단순 스케일링만 적용, validation_split 포함)
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# 훈련용 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    subset='training',      # 훈련용 데이터
    shuffle=True
)

# 검증용 데이터 생성기
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    subset='validation',    # 검증용 데이터
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    shuffle=False             # 테스트 데이터는 셔플하지 않음
)

Found 38280 images belonging to 100 classes.
Found 4211 images belonging to 100 classes.
Found 10587 images belonging to 100 classes.


In [3]:
# 간단한 모델 생성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(100, activation='softmax')  # 100개의 클래스로 분류
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
# 모델 학습
model.fit(
    train_generator,
    epochs=4,
    validation_data=validation_generator
)

Epoch 1/4
1197/1197 [==============================] - 160s 132ms/step - loss: 3.7349 - accuracy: 0.1261 - val_loss: 3.1760 - val_accuracy: 0.2080
Epoch 2/4
1197/1197 [==============================] - 157s 131ms/step - loss: 2.7985 - accuracy: 0.2866 - val_loss: 2.7964 - val_accuracy: 0.2966
Epoch 3/4
1197/1197 [==============================] - 157s 131ms/step - loss: 2.0325 - accuracy: 0.4586 - val_loss: 2.8428 - val_accuracy: 0.3018
Epoch 4/4
1197/1197 [==============================] - 157s 131ms/step - loss: 0.9635 - accuracy: 0.7293 - val_loss: 3.6653 - val_accuracy: 0.2800


In [9]:
# 테스트 세트에 대한 모델 평가
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

331/331 [==============================] - 31s 92ms/step - loss: 3.6196 - accuracy: 0.2763
Test Loss: 3.6195621490478516
Test Accuracy: 0.2762822210788727


In [11]:
model.save("naive2DCNN.h5")

In [13]:
tf.keras.backend.clear_session()

In [13]:
cnnmodel = load_model("naive2DCNN.h5")

In [15]:
# 테스트 세트에 대한 모델 평가
test_loss, test_accuracy = cnnmodel.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

331/331 [==============================] - 32s 90ms/step - loss: 3.6196 - accuracy: 0.2763
Test Loss: 3.6195621490478516
Test Accuracy: 0.2762822210788727
